# Malaria challenge trial calibration

### A history matching approach to calibrating an empirical model of malaria infection and immunity
### For iter0 we will cut down parameter space by using the distribution of infection durations by age in the KND cohort in Navrongo, Ghana

# Import block

In [1]:
%load_ext autoreload
%autoreload 2

import os, glob, re, sys, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from wand.image import Image as WImage
from IPython.display import display

from pyDOE import lhs
from history_matching import HistoryMatching, HistoryMatchingCut, quick_read, Basis

import pycuda

sns.set(font_scale=2)
sns.set_style("white")

c:\python36\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


RuntimeError: cuInit failed: no CUDA-capable device is detected

# Configure parameters and data for history matching

In [ ]:
age_bin = 0
cut_name = 'Duration for age='%age_bin
DROP_BIN_20 = True

iteration = int(re.search(r'iter(\d+)', os.getcwd()).group(1)) # Index of the current iteration
exp_ids = [''] # TODO: only uses first for now, should loop